In [1]:
print("ok")

ok


In [2]:
pip install langchain


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install -U langchain-community


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pinecone
from langchain.chains import RetrievalQA
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
import pinecone
print(pinecone.__version__)

7.3.0


In [6]:
PINECONE_API_KEY = "pcsk_3qkuRr_AT18M4hh36Z1kiVmXq6JMrJhyrJ5sZANVk2nDxzo5P67Z3sDwAvHNy767vo4S2h"
PINECONE_API_ENV = "aped-4627-b74a"

In [7]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents


In [8]:
extracted_data = load_pdf("data/")

In [9]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
len(text_chunks)

5859

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\kande\AppData\Local\Temp\ipykernel_13608\3738240079.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
from pinecone import Pinecone

# Initialize the Pinecone client
pc = Pinecone(api_key="pcsk_3qkuRr_AT18M4hh36Z1kiVmXq6JMrJhyrJ5sZANVk2nDxzo5P67Z3sDwAvHNy767vo4S2h")

# Connect to your existing index
index = pc.Index("medical-chatbot")


In [15]:
texts = [t.page_content for t in text_chunks]
embedded_texts = embeddings.embed_documents(texts)


In [16]:
from langchain.vectorstores import Pinecone as LangchainPinecone


In [17]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_3qkuRr_AT18M4hh36Z1kiVmXq6JMrJhyrJ5sZANVk2nDxzo5P67Z3sDwAvHNy767vo4S2h"


In [18]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore  # ✅ Correct import for new SDK

# Set your Pinecone API key as env var
os.environ["PINECONE_API_KEY"] = "pcsk_3qkuRr_AT18M4hh36Z1kiVmXq6JMrJhyrJ5sZANVk2nDxzo5P67Z3sDwAvHNy767vo4S2h"

# Initialize Pinecone client
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index("medical-chatbot")  # Your existing index name

# Extract texts from your chunked documents
texts = [t.page_content for t in text_chunks]

# Upload documents to Pinecone using the correct vector store
docsearch = PineconeVectorStore.from_texts(
    texts=texts,
    embedding=embeddings,        # this must be a LangChain-compatible embedding
    index_name="medical-chatbot" # this must match the name of your existing index
)


In [19]:
!pip install -U langchain-pinecone


  Attempting uninstall: langchain-pinecone
    Found existing installation: langchain-pinecone 0.2.9
    Uninstalling langchain-pinecone-0.2.9:
      Successfully uninstalled langchain-pinecone-0.2.9



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from langchain_pinecone import PineconeVectorStore

# Load the existing index
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medical-chatbot",
    embedding=embeddings
)

# Your query
query = "What are allergies?"

# Perform similarity search
docs = docsearch.similarity_search(query, k=3)

# Print results
print("Results:\n")
for i, doc in enumerate(docs, 1):
    print(f"{i}. {doc.page_content}\n")


Results:

1. reaction. Allergic rhinitis is characterized by an itchy,
runny nose, often with a scratchy or irritated throat due
to post-nasal drip. Inflammation of the thin membrane
covering the eye (allergic conjunctivitis) causes redness,
irritation, and increased tearing in the eyes. Asthma caus-
es wheezing, coughing, and shortness of breath. Symp-
toms of food allergies depend on the tissues most sensi-
tive to the allergen and whether the allergen spread sys-

2. reactions is triggered by harmless, everyday substances.
This is the condition known as allergy, and the offend-
ing substance is called an allergen. Common inhaled
allergens include pollen, dust, and insect parts from tiny
house mites. Common food allergens include nuts, fish,
and milk.
Allergic reactions involve a special set of cells in
the immune system known as mast cells. Mast cells
serve as guards in the tissues where the body meets the

3. Purpose
Allergy is a reaction of the immune system. Nor-
mally, the immun

In [21]:
from langchain.prompts import PromptTemplate


In [22]:
from langchain.prompts import PromptTemplate

# Define the prompt template string
template_string = """
You are a helpful medical assistant. Use the context below to answer the user's question.
If the answer is not in the context, say you don't know.

Context:
{context}

Question:
{questions}

Answer:
"""

# Create the PromptTemplate
PROMPT = PromptTemplate(
    template=template_string,
    input_variables=["context", "questions"]
)

# Wrap in chain_type_kwargs if needed later in a retrieval chain
chain_type_kwargs = {"prompt": PROMPT}


In [23]:
!pip install ctransformers



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained(
    "model/llama-2-7b-chat.ggmlv3.q4_0.bin",  # Path to your .bin model
    model_type="llama",                      # Model type (llama, gpt2, etc.)
    max_new_tokens=512,
    temperature=0.8
)


In [25]:
!pip install langchain-community ctransformers



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from langchain_community.llms import CTransformers

llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",  # Local path to your model
    model_type="llama",                            # Type must match (llama)
    config={
        "max_new_tokens": 512,
        "temperature": 0.8
    }
)


In [27]:
from langchain.prompts import PromptTemplate

template_string = """Use the following context to answer the question:

{context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template_string
)

chain_type_kwargs = {"prompt": PROMPT}


In [28]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs  # only if using a custom prompt
)


In [29]:
while True:
    query = input("Input Prompt: ")
    result = qa({"query": query})
    print("Response:", result["result"])


C:\Users\kande\AppData\Local\Temp\ipykernel_13608\2548574920.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": query})


KeyboardInterrupt: 